# MOFA+

In this notebook we "train" our MOFA model, in order to find the Latent Factors.

In [1]:
from mofapy2.run.entry_point import entry_point
import pandas as pd
import cupy as cp
import numpy as np

In [2]:
print(cp.cuda.runtime.getDeviceCount())
print(cp.__version__)

1
13.6.0


In [3]:
ent = entry_point()


        #########################################################
        ###           __  __  ____  ______                    ### 
        ###          |  \/  |/ __ \|  ____/\    _             ### 
        ###          | \  / | |  | | |__ /  \ _| |_           ### 
        ###          | |\/| | |  | |  __/ /\ \_   _|          ###
        ###          | |  | | |__| | | / ____ \|_|            ###
        ###          |_|  |_|\____/|_|/_/    \_\              ###
        ###                                                   ### 
        ######################################################### 
       
 
        


## Data Loading

In [4]:
GO_BP = pd.read_csv('data/gene_go_matrix_propT_rel-is_a-part_of_ont-BP.csv', index_col=0)
GO_CC = pd.read_csv('data/gene_go_matrix_propT_rel-is_a-part_of_ont-CC.csv', index_col=0)
GO_MF = pd.read_csv('data/gene_go_matrix_propT_rel-is_a-part_of_ont-MF.csv', index_col=0)
HPO = pd.read_csv('data/gene_hpo_matrix_binary_withAncestors_namespace_Phenotypic abnormality.csv', index_col=0)

In [5]:
GO_BP = GO_BP.values.astype(float)
GO_CC = GO_CC.values.astype(float)
GO_MF = GO_MF.values.astype(float)
HPO = HPO.values.astype(float)

In [6]:
print(GO_BP.shape)
print(GO_CC.shape)
print(GO_MF.shape)
print(HPO.shape)

(5183, 9873)
(5183, 1478)
(5183, 3258)
(5183, 10185)


In [7]:
D = [GO_BP.shape[1],GO_CC.shape[1], GO_MF.shape[1], HPO.shape[1]] # Number of features per view
M = len(D)      # Number of views
K = 15           # Number of factors
N = [5183] * 4   # Number of samples per group
G = len(N)      # Number of groups

In [8]:
data = [[GO_BP], [GO_CC], [GO_MF], [HPO]]

## Model Training

In [9]:
ent.set_data_options(scale_groups=False, scale_views=False)
ent.set_data_matrix(
    data=data,
    likelihoods=['bernoulli', 'bernoulli', 'bernoulli', 'bernoulli']
)
ent.set_model_options(factors=15)

View names not provided, using default naming convention:
- view1, view2, ..., viewM

Features names not provided, using default naming convention:
- feature1_view1, featureD_viewM

Groups names not provided, using default naming convention:
- group1, group2, ..., groupG

Samples names not provided, using default naming convention:
- sample1_group1, sample2_group1, sample1_group2, ..., sampleN_groupG

Successfully loaded view='view0' group='group0' with N=5183 samples and D=9873 features...
Successfully loaded view='view1' group='group0' with N=5183 samples and D=1478 features...
Successfully loaded view='view2' group='group0' with N=5183 samples and D=3258 features...
Successfully loaded view='view3' group='group0' with N=5183 samples and D=10185 features...


Model options:
- Automatic Relevance Determination prior on the factors: False
- Automatic Relevance Determination prior on the weights: True
- Spike-and-slab prior on the factors: False
- Spike-and-slab prior on the weights: Tr

In [14]:
ent.set_train_options(iter=100, convergence_mode='fast', gpu_mode=True)


GPU mode is activated



In [15]:
ent.build()
ent.run()



######################################
## Training the model with seed 808855 ##
######################################


ELBO before training: -267670991.78 

Iteration 1: time=10.67, ELBO=-138489316.80, deltaELBO=129181674.980 (48.26136524%), Factors=15
Iteration 2: time=10.54, ELBO=-88917803.67, deltaELBO=49571513.133 (18.51956867%), Factors=15
Iteration 3: time=10.48, ELBO=-88788966.94, deltaELBO=128836.734 (0.04813250%), Factors=15
Iteration 4: time=10.50, ELBO=-88768947.86, deltaELBO=20019.080 (0.00747899%), Factors=15
Iteration 5: time=10.47, ELBO=-88753344.91, deltaELBO=15602.948 (0.00582915%), Factors=15
Iteration 6: time=10.52, ELBO=-88734690.65, deltaELBO=18654.260 (0.00696910%), Factors=15
Iteration 7: time=10.66, ELBO=-88717016.70, deltaELBO=17673.947 (0.00660286%), Factors=15
Iteration 8: time=10.47, ELBO=-88701896.22, deltaELBO=15120.478 (0.00564890%), Factors=15
Iteration 9: time=10.45, ELBO=-88688746.86, deltaELBO=13149.360 (0.00491251%), Factors=15
Iteration 10: tim

## Saving the Model

In [16]:
ent.save(outfile='data/mofa.hdf5', save_data=True)

Saving model in data/mofa.hdf5...
